In [1]:
import cv2
import numpy as np
import math
from skimage.draw import line

In [2]:
# Req 1

def load_map(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    occupancy_grid = (img < 100).astype(int)
    return occupancy_grid

def generate_measurements(path, robot_pose):
    x, y, theta = robot_pose
    # Read the map image
    map = load_map(path)
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    
    # Draw Robot
    img = cv2.circle(img, (x, y), 10, (0, 0, 255), -1)
    
    # Initialize distances which will represent the laser measurements
    distances = np.zeros(int(250 / 2) + 1)
    
    # Determine the range of angles to be scanned by the robot
    starting_angle = theta - int(250 / 2)
    finishing_angle = theta + int(250 / 2)
    angles = np.arange(starting_angle, finishing_angle + 1, 2)
    
    # Calculate max reachable length
    max_length = int(1200 / 4)
    # Cast rays
    for i, angle in enumerate(angles):
        angle_rad = np.radians(angle)
        x_end = int(x + (max_length * math.cos(angle_rad)))
        y_end = int(y + (max_length * math.sin(angle_rad)))
        points = list(zip(*line(x, y, x_end, y_end)))
        endpoint = None
        for point in points:
            X, Y = point
            endpoint = point
            if map[Y, X] == 1:
                distance_traveled = np.linalg.norm(np.array((x, y)) - np.array(point))
                distances[i] = min(1200, distance_traveled * 4)
                break
        if distances[i] == 0:
            distances[i] = 1200
        img = cv2.line(img, (x, y), endpoint, (0, 255, 0), thickness=1)
    cv2.imwrite('Measurements.jpg', img)
    return distances

In [3]:
distances = generate_measurements("Map.jpg", (350, 185, 200))
print(distances)

[ 877.08608471  870.3562489   863.71291527  858.212095    854.35355679
  546.11354131  544.72011162  544.05882035  544.13233684  849.14074216
  510.26267745  475.79827658  478.0794913   480.88252204  484.19830648
  489.0480549   493.47745643  499.67989753  506.59648637  512.624619
  520.75330052  531.3379339   802.28673179  757.34272295  716.70356494
  680.29405407  111.2115102   108.07404869  101.98039027  104.69001863
  107.48023074  110.34491379  116.27553483  122.44182292  125.60254774
  132.06059215  138.67948659  145.4372717   155.79473675  166.37307474
  177.1327186   191.70811146  210.2189335   232.75738442  259.38388539
  290.13100489  332.79423072  391.44859177  485.97942343  620.67060507
  891.24631837 1200.         1200.         1200.         1200.
 1184.93206556  798.28816852  603.68203551  534.22092808  640.39987508
  364.85613603  322.61432082  292.21909589  265.93232222  243.70473939
  225.45953074  207.53794834  196.97715604  186.59046064  176.40861657
  166.46921637  

In [4]:
# Req 2

def read_map(path):
    map = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    return map, cv2.threshold(map, 127, 255, cv2.THRESH_BINARY)[1]

def generate_likelihood(img):
    return np.array(cv2.distanceTransform(img, cv2.DIST_L2, 0))

def apply_gaussian(likelihood_field, sigma):
    return 1 / np.sqrt(2 * np.pi * sigma) * np.exp(-0.5 * (likelihood_field / sigma)**2)

def apply_random_noise(likelihood_field):
    return 0.9 * likelihood_field + 0.05 / (300 * np.max(likelihood_field))

def normalize(likelihood_field):
    return likelihood_field / np.max(likelihood_field)

def save_likelihood(likelihood_field):
    cv2.imwrite("Likelihood_field.jpg", likelihood_field * 255)
    
def save_result(highest_likelihood):
    cv2.imwrite("Likelihood.jpg", highest_likelihood * (255 / np.max(highest_likelihood)))
    
def get_endpoint(x, y, r, theta):
    return int(x + (r // 0.04) * math.cos(np.radians(theta))), int(y + (r // 0.04) * math.sin(np.radians(theta)))

def out_of_bounds(x, y, L):
    return x < 0 or x >= L.shape[1] or y < 0 or y >= L.shape[0]

def get_highest_likelihood(map, likelihood, distance):
    # Initialize Probability map
    highest_likelihood = np.zeros((len(map), len(map[0])))
    
    Y = likelihood.shape[0]
    X = likelihood.shape[1]
    
    for y in range(Y):
        for x in range(X):
            for theta in range(360):
                probability = 1
                x_end, y_end = get_endpoint(x, y, distance, theta)        
                probability *= 0.001 if out_of_bounds(x_end, y_end, likelihood) else likelihood[y_end][x_end]
                highest_likelihood[y][x] = max(highest_likelihood[y][x], probability)
    
    return highest_likelihood

def compute_likelihood(path):
    map, binary_map = read_map(path)
    
    likelihood = generate_likelihood(binary_map)
    
    likelihood = apply_gaussian(likelihood, sigma=1)
    
    likelihood = normalize(likelihood)

    save_likelihood(likelihood)

    likelihood = apply_random_noise(likelihood)

    highest_likelihood = get_highest_likelihood(map, likelihood, distance=0.1)
                    
    save_result(highest_likelihood)

In [ ]:
compute_likelihood("Map.jpg")